In [32]:
import pickle
trafficRecordsByIdLoaded = pickle.load(open("trafficRecordsById.p", "rb"))

In [33]:
def intervalToRecords(trafficRecordsForId):
    intervalToRecordsMap = {}
    if trafficRecordsForId is None:
        return intervalToRecordsMap
    for trafficRecord in trafficRecordsForId:
        if sum(trafficRecord[1]) < 400:
            intervalToRecordsMap[int(trafficRecord[0])] = sum(trafficRecord[1])
    return intervalToRecordsMap

In [34]:
def trafficRecordsMap():
    toReturn = {}
    for sensorId, records in trafficRecordsByIdLoaded.items():
        toReturn[sensorId] = intervalToRecords(records)
    return toReturn

In [42]:
import csv
import numpy as np
mapeAvg = {}
mapeList = {}
tMap = trafficRecordsMap()
with open("singlePointTestUnder400.csv") as f:
    predicted = csv.reader(f, delimiter=',')
    for p in predicted:
        sensorId = p[0]
        if sensorId not in mapeList:
            mapeList[sensorId] = [[], [], [], [], [], []]
        intervalId = int(p[1])
        if not (intervalId % 288 > 91 and intervalId % 288 < 102 or intervalId % 288 > 205 and intervalId % 288 < 215):
            continue
        if not (tMap.get(sensorId)):
            continue
        for i in range(6):
            trueValue = tMap.get(sensorId).get(intervalId + i)
            predictedValue = float(p[len(p) - 1 - i])
            if trueValue is None or predictedValue is None or trueValue == 0:
                continue
            mapeList[sensorId][i].append(np.abs((trueValue - predictedValue) / trueValue * 100))

with open("mape6stepsPeakHourUnder400.csv", "w") as f:
    for sensorId, l in mapeList.items():
        toWrite = [sensorId]
        for i in range(6):
            if not l[i]:
                toWrite.append("None")
                continue
            toWrite.append(str(sum(l[i])/len(l[i])))
        f.write(",".join(toWrite) + "\n")        

In [66]:
import csv
import numpy as np
tMap = trafficRecordsMap()
resultMap = {}
with open("surroundingTestWithBadValueExcludedUnder400.csv") as f:
    predicted = csv.reader(f, delimiter=',')
    for p in predicted:
        sensorId, intervalId, predictedValue = p
        intervalId = int(intervalId)
        if not (intervalId % 288 > 73 and intervalId % 288 < 276):
            continue
        predictedValue = float(predictedValue)
        trueValue = tMap.get(sensorId).get(intervalId)

        if not trueValue:
            continue
        if sensorId not in resultMap:
            resultMap[sensorId] = []
        resultMap[sensorId].append(np.abs((trueValue - predictedValue) / trueValue * 100))
        
with open("./mapeSurrounding/mapeSurroundingTestWithBadValueExcludedUnder400DayTime.csv", "w") as f:
    for sensorId, mape in resultMap.items():
        toWrite = [sensorId]
        toWrite.append(str(sum(mape)/len(mape)))
        f.write(",".join(toWrite) + "\n")
        

In [75]:
sensorSelected = {"101",
"102",
"104",
"111",
"203",
"206",
"307",
"405",
"406",
"407",
"408",
"503",
"504",
"505",
"513",
"604",
"704",
"804"}

In [82]:
result = {}
with open("mape6stepsPeakHourUnder400.csv") as f:
    mape = csv.reader(f, delimiter=',')
    for p in mape:
        if p[0] not in sensorSelected:
            continue
        for i in range(6):
            if i not in result:
                result[i] = []
#             print (p[i + 1])
            result[i].append(float(p[i + 1]))
for i in range(6):
    result[i] = sorted(result[i])
    print (np.percentile(result[i], 20), np.percentile(result[i], 80), np.mean(result[i]))

8.97969453149 12.6236611102 10.8562455107
9.98100138445 13.9048648395 11.8693719848
10.7469773712 14.2168675607 12.4068813545
11.4107748228 14.7407016752 13.2690385428
12.1252361288 16.0051756122 13.9813982402
11.9827852547 16.7775714751 14.6380405632


In [96]:
l = []
with open("./mapeSurrounding/mapeSurroundingTestWithBadValuesUnder400PeakHour.csv") as f:
    mape = csv.reader(f, delimiter=',')
    for p in mape:
        if p[0] not in sensorSelected:
            continue
        l.append(float(p[1]))
print (np.percentile(l, 20), np.percentile(l, 80), np.mean(l))

8.15747363703 11.7172854991 10.0251419495
